### Imports

In [1]:
import tweepy
import credentials
import psycopg2
import time

In [1]:
# DB Connection test

# conn = psycopg2.connect(host="localhost",database="twitter",port=5432,user="postgres",password="abstract")
# conn.close

### Credentials

In [2]:
ckey = credentials.consumer_key
csecret = credentials.consumer_secret
atoken = credentials.access_token
asecret = credentials.access_token_secret

# db credentials
#user_name = credentials.user
#password = credentials.password

### Streaming

In [3]:
# Insert Tweet data into database
def dbConnect(user_id, user_name, tweet_id, tweet, retweet_count, hashtags):
    
    conn = psycopg2.connect(host="localhost",database="twitter",port=5432,user=credentials.user ,password=credentials.password)
    
    cur = conn.cursor()

    # insert user information
    command = '''INSERT INTO TwitterUser (user_id, user_name) VALUES (%s,%s) ON CONFLICT
                 (User_Id) DO NOTHING;'''
    cur.execute(command,(user_id,user_name))

    # insert tweet information
    command = '''INSERT INTO TwitterTweet (tweet_id, user_id, tweet, retweet_count) VALUES (%s,%s,%s,%s);'''
    cur.execute(command,(tweet_id, user_id, tweet, retweet_count))
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        #print(hashtag)
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag["text"]))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()

In [4]:
# Subclass tweepy.Stream to print the tweets
class TweetPrinter(tweepy.Stream):
    def on_status(self, status):
        # Tweet ID
        tweet_id = status.id
        # User ID
        user_id = status.user.id
        # Username
        username = status.user.name
        # text
        tweet = status.text
        # Retweet count
        retweet_count = status.retweet_count
        # Language
        lang = status.lang

        # Tweet
        if status.truncated == True:
            tweet = status.extended_tweet['full_text']
            hashtags = status.extended_tweet['entities']['hashtags']
            #hashtags = status.hashtags
        else:
            tweet = status.text
            hashtags = status.entities['hashtags']

        
        # If tweet is not a retweet and tweet is in English
        if not hasattr(status, "retweeted_status") and lang=="en":
            # Connect to database
            dbConnect(user_id, username, tweet_id, tweet, retweet_count, hashtags)
            
 
# Initialize instance of the subclass
printer = TweetPrinter(
  ckey, csecret,
  atoken, asecret
)

In [13]:
round(time.time())

1653218333

In [ ]:
while True:
    printer.filter(track=["monkeypox"], locations=[33.958740,-2.602864, 42.198486,4.291636])
    time.sleep(60)


In [21]:
#printer.filter(track=["karua"])

# bounding box for kenya
# 33.958740,-2.602864, 42.198486,4.291636

start_time = round(time.time())
print("Stream started")

while round(time.time()) > (start_time + 300):
    printer.filter(track=["monkeypox"])
    #printer.filter(locations=[33.958740,-2.602864, 42.198486,4.291636])

print("Stream ended")


Stream started
Stream ended
